In [3]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
from addFeatures2Cand import ADD_covWgt, ADD_cfSim, Add_features, ADD_w2vSim, ADD_cfSim_lastN, ADD_w2vSim_lastN, ADD_covScore_lastN, ADD_covWgt_cnt, Add_freq_features
import os
from annoy import AnnoyIndex

In [4]:
TOPN_candidate = 100
SETS = [2, 3]
predTypes = ['orders']
SUBSETNUM = 2
input_note = 'covisit_20_20_20_newSuggester2'

### user, item -based features

In [ ]:
add_features = Add_features(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note)
add_features.process()

In [5]:
add_features = Add_freq_features(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note)
add_features.process()

set: 2
[Errno 17] File exists: '../output/newSplited/candidates/set2_top_100/covisit_20_20_20_newSuggester2_features_norm_freq_t0/'
type: orders
sub:  0
sub:  1
set: 3
[Errno 17] File exists: '../output/newSplited/candidates/set3_top_100/covisit_20_20_20_newSuggester2_features_norm_freq_t0/'
type: orders
sub:  0
sub:  1


### covisitation feature

In [8]:
add_features = ADD_covWgt(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note, True)
add_features.process()

set: 2
type: orders
sub:  0
sub:  1
sub:  2
sub:  3
sub:  4
sub:  5
set: 3
type: orders
sub:  0
sub:  1
sub:  2
sub:  3
sub:  4
sub:  5


In [ ]:
# add_features = ADD_covWgt_cnt(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note, False)
# add_features.process()

In [ ]:
for i in range(1, 2):
    add_features = ADD_covScore_lastN(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note, False)
    add_features.lastN = i
    add_features.output_note = f'{add_features.output_note}_{add_features.lastN}'
    add_features.process()

### CF features

In [4]:
add_features = ADD_cfSim(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note, True)
add_features.output_note = 'cfSim'
add_features.process()

set: 2


A Jupyter Widget

type: orders
sub:  0
sub:  1
sub:  2
sub:  3
sub:  4
sub:  5
set: 3


A Jupyter Widget

type: orders
sub:  0
sub:  1
sub:  2
sub:  3
sub:  4
sub:  5


In [ ]:
for i in range(1, 4):
    add_features = ADD_cfSim_lastN(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note, False)
    add_features.lastN = i
    add_features.output_note = f'{add_features.output_note}_{add_features.lastN}'
    add_features.process()

### w2v features

In [ ]:
add_features = ADD_w2vSim(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note, True)
add_features.output_note = 'w2vSim_3x'
add_features.process()

In [ ]:
for i in range(1, 5):
    add_features = ADD_w2vSim_lastN(TOPN_candidate, SETS, predTypes, SUBSETNUM, input_note, False)
    add_features.lastN = i
    add_features.output_note = f'{add_features.output_note}_{add_features.lastN}'
    add_features.process()

## Merge candidate & features

### Candidates to data4xgb

In [3]:
def mergeFeatures(feature_notes, candidates_path, predType, s, data4xgb_path):
    for e, feature_note in enumerate(feature_notes):
        if e == 0:
            data4xgb = pd.read_parquet(f'{candidates_path}/{predType}_{s}.pqt').reset_index(drop=True).astype('int32')
        else:
            thisFeature =  pd.read_parquet(f'{candidates_path}_{feature_note}/{predType}_{s}.pqt').iloc[:, 2:].astype('float32').reset_index(drop=True)
            data4xgb = pd.concat([data4xgb, thisFeature], axis=1)
    data4xgb = data4xgb.fillna(0)
    print(data4xgb.shape)
    data4xgb.to_parquet(f'{data4xgb_path}/{predType}_{s}.pqt')

In [4]:
outputPath = '../output/newSplited/'
# feature_notes = ['', 'features_dropped', 'covWgt_t1', 'cfSim_last_1_t0', 'cfSim_last_2_t0', 'cfSim_last_3_t0', 'w2vSim_last_1_t0', 'w2vSim_last_2_t0', 'w2vSim_last_3_t0', 'w2vSim_last_4_t0', 'covScore_last_1_t0']
feature_notes = ['', 'features_dropped', 'covWgt_t1', 'covScore_last_1_t0'] #'cfSim_last_1_t0', 'cfSim_last_2_t0', 'cfSim_last_3_t0', 'w2vSim_last_1_t0', 'w2vSim_last_2_t0', 'w2vSim_last_3_t0', 'w2vSim_last_4_t0', ]
data4xgb_note = 'suggester_addLast'

for SET in SETS:
    candidates_path = outputPath + f'candidates/set{SET}_top_{TOPN_candidate}/{input_note}'
    data4xgb_path = outputPath + f'data4xgb/set{SET}_top_{TOPN_candidate}/{data4xgb_note}'
    try: 
        os.makedirs(data4xgb_path) 
    except OSError as error: 
        print(error)

    for predType in predTypes:
        for s in range(SUBSETNUM):
            print(SET, predType, s)
            mergeFeatures(feature_notes, candidates_path, predType, s, data4xgb_path)

[Errno 17] File exists: '../output/newSplited/data4xgb/set2_top_100/suggester_addLast'
2 orders 0
(33000000, 36)
2 orders 1
(33000000, 36)
2 orders 2
(33000000, 36)
2 orders 3
(33000000, 36)
2 orders 4
(33000000, 36)
2 orders 5
(15125100, 36)
[Errno 17] File exists: '../output/newSplited/data4xgb/set3_top_100/suggester_addLast'
3 orders 0
(33000000, 35)
3 orders 1
(33000000, 35)
3 orders 2
(33000000, 35)
3 orders 3
(33000000, 35)
3 orders 4
(33000000, 35)
3 orders 5
(2180324, 35)


### data4xgb to data4xgb

In [6]:
def addFeatures(feature_notes, input_path, predType, s, data4xgb_path, output_path):
    # if predType == 'orders':
        # toDrop = ['covScore_cartsOrders_2', 'covScore_buy2buy_2', 'covScore_cartsOrders_3', 'covScore_buy2buy_3', 'covScore_buy2buy_1', 'item_ts_min_valA', 'item_type_std_valA']
    data4xgb = pd.read_parquet(f'{data4xgb_path}/{predType}_{s}.pqt').reset_index(drop=True)#.drop(columns=toDrop)
    
    for feature_note in feature_notes:
        thisFeature =  pd.read_parquet(f'{input_path}_{feature_note}/{predType}_{s}.pqt').iloc[:, 2:].astype('float32').reset_index(drop=True)
        data4xgb = pd.concat([data4xgb, thisFeature], axis=1)
    data4xgb.fillna(0)
    print(data4xgb.shape)
    # data4xgb.to_parquet(f'{output_path}/{predType}_{s}.pqt')

    chunkSize = 33000000
    idx = 0
    n = len(data4xgb)
    while (idx * chunkSize < n):
        # dtest = xgb.DMatrix(data4xgb.iloc[i*chunkSize:(i+1)*chunkSize, 2:])
        sub = data4xgb[idx*chunkSize:(idx+1)*chunkSize].reset_index(drop=True)
        sub.to_parquet(f'{output_path}/{predType}_{s}_{idx}.pqt')
        idx += 1

In [7]:
outputPath = '../output/newSplited/'
# feature_notes = ['cfSim_3x_t1', 'cfSim_last_1_t0', 'cfSim_last_2_t0', 'cfSim_last_3_t0'] 
# feature_notes =  ['w2vSim_3x_t1', 'w2vSim_last_1_t0', 'w2vSim_last_2_t0', 'w2vSim_last_3_t0', 'w2vSim_last_4_t0']
feature_notes = ['features_norm_freq_t0']
data4xgb_note = 'covisit_20_20_20_newSuggester2_drop_12_add_last3CovScore'
output_note = 'covisit_20_20_20_newSuggester2_add_freq'
for SET in SETS:
    input_path = outputPath + f'candidates/set{SET}_top_{TOPN_candidate}/{input_note}'
    data4xgb_path = outputPath + f'data4xgb/set{SET}_top_{TOPN_candidate}/{data4xgb_note}'
    output_path = outputPath + f'data4xgb/set{SET}_top_{TOPN_candidate}/{output_note}'
    try: 
        os.makedirs(output_path) 
    except OSError as error: 
        print(error)

    for predType in predTypes:
        for s in range(SUBSETNUM):
            print(SET, predType, s)
            addFeatures(feature_notes, input_path, predType, s, data4xgb_path, output_path)

[Errno 17] File exists: '../output/newSplited/data4xgb/set2_top_100/covisit_20_20_20_newSuggester2_add_freq'
2 orders 0
(90062550, 50)
2 orders 1
(90062550, 50)
[Errno 17] File exists: '../output/newSplited/data4xgb/set3_top_100/covisit_20_20_20_newSuggester2_add_freq'
3 orders 0


KeyboardInterrupt: 

### Drop columns from data4xgb

In [3]:
input_note = 'suggester_addLast'
output_note = 'suggester_addLast'

for SET in [2, 3]:
    ! mkdir ../output/newSplited/data4xgb/set{SET}_top_100/{output_note}
    for predType in predTypes:
        if predType == 'clicks':
            toDrop_col = ['item_type_median', 'item_type_median_valA', 'item_clicked_cnt_val', 'item_carted_cnt_val', 'item_ordered_cnt_val', 'user_type_median']
        elif predType == 'carts':
            toDrop_col = ['item_type_median', 'item_type_median_valA', 'item_clicked_cnt_val', 'item_carted_cnt_val', 'item_ordered_cnt_val', 'user_type_median']
        elif predType == 'orders':
            # toDrop_col = ['item_type_median', 'item_type_median_valA', 'item_clicked_cnt_val', 'item_carted_cnt_val', 'item_ordered_cnt_val', 'user_type_median']
            # toDrop_col = ['item_type_median_valA', 'item_type_median', 'user_type_median', 'item_clicked_cnt_val', 'item_ordered_cnt_val']
            toDrop_col = ['covScore_buy2buy_1', 'wgt_buy2buy', 'user_ts_diff_std', 'item_carted_cnt_val', 'user_ts_min']

            
        for sub in range(SUBSETNUM):
            data4xgb = pd.read_parquet(f'../output/newSplited/data4xgb/set{SET}_top_100/{input_note}/{predType}_{sub}.pqt').drop(toDrop_col, axis=1)
            data4xgb.to_parquet(f'../output/newSplited/data4xgb/set{SET}_top_100/{output_note}/{predType}_{sub}.pqt')

mkdir: cannot create directory ‘../output/newSplited/data4xgb/set2_top_100/suggester_addLast’: File exists
mkdir: cannot create directory ‘../output/newSplited/data4xgb/set3_top_100/suggester_addLast’: File exists


### Drop columns from candidates

In [4]:
def load_toDrop_col(predType):
    if predType == 'clicks':
        toDrop_col = ['user_ordered_cnt', 'cnt_ordered', 'user_lastAid', 'user_clicked_cnt', 'user_carted_cnt', 'item_ts_min_valA', 'user_type_std', 'user_ts_min', 'item_buy_ratio_valA', 'item_type_std_valA', 'item_buy_ratio', 'item_type_std']
    elif predType == 'carts':
        toDrop_col = ['user_ordered_cnt', 'cnt_ordered', 'user_clicked_cnt', 'item_item_count', 'user_user_count', 'item_carted_cnt', 'user_type_std', 'item_user_count_valA', 'user_ts_min', 'item_ordered_cnt', 'item_ts_min_valA', 'user_lastAid']
    elif predType == 'orders':
        toDrop_col = ['user_ordered_cnt', 'item_user_count_valA', 'user_lastAid', 'item_item_count', 'user_ts_mean', 'user_clicked_cnt', 'item_clicked_cnt', 'item_item_count_valA', 'user_type_std', 'item_ts_min', 'user_user_count', 'item_ts_min_valA', 'item_type_std_valA']
    return toDrop_col

In [5]:
input_note = 'features_norm_addLast_t0'
output_note = 'features_dropped'

for SET in [2, 3]:
    ! mkdir ../output/newSplited/candidates/set{SET}_top_100/suggester_addLast_{output_note}
    for predType in predTypes:
        toDrop_col = load_toDrop_col(predType)
        
        for sub in range(SUBSETNUM):
            data4xgb = pd.read_parquet(f'../output/newSplited/candidates/set{SET}_top_100/suggester_addLast_{input_note}/{predType}_{sub}.pqt').drop(toDrop_col, axis=1)
            data4xgb.to_parquet(f'../output/newSplited/candidates/set{SET}_top_100/suggester_addLast_{output_note}/{predType}_{sub}.pqt')

## user-based & item-based features selection

In [3]:
toDrop_col = ['item_type_median', 'item_type_median_valA', 'item_clicked_cnt_val', 'item_carted_cnt_val', 'item_ordered_cnt_val', 'user_type_median']

for SET in [2, 3]:
    ! mkdir ../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0
    for predType in ['clicks', 'carts', 'orders']:
        for sub in range(2):
            data4xgb = pd.read_parquet(f'../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_addLast_t0/{predType}_{sub}.pqt').drop(toDrop_col, axis=1)
            data4xgb.to_parquet(f'../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0/{predType}_{sub}.pqt')

mkdir: cannot create directory ‘../output/newSplited/candidates/set2_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0’: File exists
mkdir: cannot create directory ‘../output/newSplited/candidates/set2_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0’: File exists
mkdir: cannot create directory ‘../output/newSplited/candidates/set2_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0’: File exists
mkdir: cannot create directory ‘../output/newSplited/candidates/set2_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0’: File exists
mkdir: cannot create directory ‘../output/newSplited/candidates/set2_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0’: File exists
mkdir: cannot create directory ‘../output/newSplited/candidates/set3_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0’: File exists
mkdir: cannot create directory ‘../output/newSplited/candidates/set3_top_100

## add item similarity with last item

In [2]:
toConcat = []
for SET in [2, 3]:
    for predType in ['clicks', 'carts', 'orders']:
        for sub in range(2):
            print(SET, predType, sub)
            data4xgb = pd.read_parquet(f'../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0/{predType}_{sub}.pqt')[['aid', 'user_lastAid']]
            toConcat.append(data4xgb)
data4xgb = pd.concat(toConcat, axis=0, ignore_index=True)
data4xgb = data4xgb.drop_duplicates().reset_index(drop=True)
data4xgb

2 clicks 0
2 clicks 1
2 carts 0
2 carts 1
2 orders 0
2 orders 1
3 clicks 0
3 clicks 1
3 carts 0
3 carts 1
3 orders 0
3 orders 1


,aid,user_lastAid
0,11830,11830.0
1,588923,11830.0
2,1732105,11830.0
3,571762,11830.0
4,884502,11830.0
...,...,...
196637084,868327,519105.0
196637085,406358,519105.0
196637086,269257,519105.0
196637087,272744,519105.0


In [3]:
import gensim
aid2vec_model = gensim.models.KeyedVectors.load_word2vec_format('../output/newSplited/savedModel/otto_aid2vec_5d.bin', binary=True)
cf_itemSimMatrix = np.load(f'../output/newSplited/cf_matrix/set_{2}/itemSimMatrix.npy', allow_pickle='TRUE').item()

tqdm.pandas()
def cf_smilarity(x):
    return aid2vec_model.similarity(str(x.aid), str(x.user_lastAid))
    # try:
    #     return [cf_itemSimMatrix[x.aid][x.user_lastAid], aid2vec_model.similarity(str(x.aid), str(x.user_lastAid))]
    # except:
    #     return [0, aid2vec_model.similarity(str(x.aid), str(x.user_lastAid))]

In [ ]:
data4xgb['user_lastAid'] = data4xgb['user_lastAid'].astype('int32')
data4xgb['w2v_sim_last'] = data4xgb.progress_apply(lambda x: cf_smilarity(x), axis=1)

# data4xgb[['cf_sim_last', 'w2v_sim_last']] = pd.DataFrame(data4xgb.progress_apply(lambda x: cf_smilarity(x), axis=1).to_list())
data4xgb

In [ ]:
data4xgb.to_parquet('../output/newSplited/features/aid2aid_similarity.pqt')

In [ ]:
for SET in [2, 3]:
    ! mkdir ../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_addLastSim_t0
    for predType in ['clicks', 'carts', 'orders']:
        for sub in range(2):
            print(SET, predType, sub)
            candidates = pd.read_parquet(f'../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0/{predType}_{sub}.pqt')
            candidates = candidates.merge(data4xgb, on=['aid', 'user_lastAid'], how='left')
            candidates.to_parquet(f'../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_addLastSim_t0/{predType}_{sub}.pqt')

In [3]:
import gensim
aid2vec_model = gensim.models.KeyedVectors.load_word2vec_format('../output/newSplited/savedModel/otto_aid2vec_5d.bin', binary=True)
tqdm.pandas()

def cf_smilarity(x):
    try:
        return [cf_itemSimMatrix[x.aid][x.user_lastAid], aid2vec_model.similarity(str(x.aid), str(x.user_lastAid))]
    except:
        return [0, aid2vec_model.similarity(str(x.aid), str(x.user_lastAid))]

for SET in [2, 3]:
    ! mkdir ../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_addLastSim_t0
    print(SET)
    cf_itemSimMatrix = np.load(f'../output/newSplited/cf_matrix/set_{SET}/itemSimMatrix.npy', allow_pickle='TRUE').item()
    for predType in ['carts', 'orders', 'clicks']:
        print(predType)
        for sub in range(2):
            print(sub)
            data4xgb = pd.read_parquet(f'../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_cutBadFeatures_t0/{predType}_{sub}.pqt')

            aidx_aidy = data4xgb[['aid', 'user_lastAid']].drop_duplicates().reset_index(drop=True)
            aidx_aidy['user_lastAid'] = aidx_aidy['user_lastAid'].astype('int32')
            aidx_aidy[['cf_sim_last', 'w2v_sim_last']] = pd.DataFrame(aidx_aidy.progress_apply(lambda x: cf_smilarity(x), axis=1).to_list())
            data4xgb = data4xgb.merge(aidx_aidy, on = ['aid', 'user_lastAid'], how='left')

            data4xgb.to_parquet(f'../output/newSplited/candidates/set{SET}_top_100/covisit_20_20_20_newSuggester2_features_norm_addLastSim_t0/{predType}_{sub}.pqt')

mkdir: cannot create directory ‘../output/newSplited/candidates/set2_top_100/covisit_20_20_20_newSuggester2_features_norm_addLastSim_t0’: File exists
2
carts
0


A Jupyter Widget

1


A Jupyter Widget

orders
0


...

A Jupyter Widget

clicks
0


A Jupyter Widget

1


A Jupyter Widget